<a href="https://colab.research.google.com/github/jzhou517/TEWH-Seizure-Project/blob/Convolutional-Autoencoder/DCAE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deep Convolutional Autoencoder - CHB-MIT Data (Daoud et. al, 2019)
## Rationale and Pseudocode

Encoder:


      Conv1d (relu activation function, filter size...)
      MaxPooling
      Conv1d
      MaxPooling
      Conv1d
      MaxPooling
      Conv1d


Decoder:


      Deconv
      Upsample
      Deconv
      Upsample
      Deconv
      Upsample


Compile via RMSprop optimizer and mean square error loss function.

4. Reconstruct EEG segments

# Code for DCAE

Implementing what was detailed above according to Daoud 2019.

## Importing Modules

In [ ]:
import tensorflow as tf
import tensorflow.keras as keras
from keras.layers import Conv1D
from keras.layers import MaxPooling1D
from keras.layers import MaxPooling2D
from keras.layers import BatchNormalization
from keras.layers import UpSampling1D
from keras.models import Sequential
from keras import layers
from keras.layers import Input, Dense, Add
from tensorflow.keras.layers import Input, Conv1D, ReLU, BatchNormalization,\
                                    Add, AveragePooling2D, Flatten, Dense
from tensorflow.keras.models import Model
import numpy as np

## Making Model

### Encoder and Decoder - Training

* Updated version uses:
  * Andrew's advice
  * New Daoud paper we found for DCAE + Bi-LSTM
    * https://ieeexplore.ieee.org/abstract/document/8598447

Residual connections can help with overfitting. Lets say y defines the layer output, x the layer input, and W the convolution, a normal convolution takes the form of: y = W*x.

A residual convolution is just: y = W*x + x

If W changes the number of layers from c_in to c_out and a kernel size of K, define a second operator U that also takes c_in to c_out but does so with a kernel size of 1. Note, U is just matrix multiplication along the feature dimension. y = Wx + Ux.


In [ ]:
def resadd(x: Tensor, y: Tensor) -> Tensor:
    out = Add()([x, y])
    return out

In [ ]:
time_length = 1280; ecg_channels = 23; in_ch = ecg_channels; out_ch = 46;
from keras import layers


input_thingy=keras.Input(shape=(time_length, in_ch))
x = layers.Conv1D(filters=out_ch, kernel_size=3, padding="same", strides=1,activation='relu')(input_thingy) 

x= layers.BatchNormalization(axis=-1)(x)
x= layers.MaxPooling1D(pool_size = 2, padding = 'valid')(x)
in_ch = out_ch;
out_ch *= 2
skip = layers.Conv1D(filters=out_ch, kernel_size=3, padding="same", strides=1,activation='relu')(x)

x= layers.BatchNormalization(axis=-1)(x)
x= layers.MaxPooling1D(pool_size = 2, padding = 'valid')(x)
in_ch = out_ch;
out_ch *= 2
x = layers.Conv1D(filters=out_ch, kernel_size=3, padding="same", strides=1,activation='relu')(x)

x= layers.BatchNormalization(axis=-1)(x)
bottleneck= layers.MaxPooling1D(pool_size = 2, padding = 'valid')(x)
in_ch = out_ch;
out_ch *= 2
x=layers.Conv1D(filters=out_ch, kernel_size=3, padding="same", strides=1)(x)
  
  # Your encoded product is of size: Batch x Time_Length / (2**network_depth) x out_ch * 2**(network_depth-1)



out_ch = out_ch / 4

x=layers.Conv1D(filters=out_ch, kernel_size=3, padding="same", strides=1)(x)

x=layers.ReLU()(x)
x=layers.BatchNormalization(axis=-1)(x)
in_ch = out_ch;
out_ch = out_ch // 2
x=layers.UpSampling1D(size=2)(x)
x=layers.Conv1D(filters=out_ch, kernel_size=3, padding="same", strides=1)(x)


x=layers.ReLU()(x)
x=layers.BatchNormalization(axis=-1)(x)
in_ch = out_ch;
out_ch = out_ch // 2
x=layers.UpSampling1D(size=2)(x)
x=layers.Conv1D(filters=out_ch, kernel_size=3, padding="same", strides=1)(x)

x=layers.ReLU()(x)
x=layers.BatchNormalization(axis=-1)(x)
in_ch = out_ch;
out_ch = out_ch // 2
x=layers.UpSampling1D(size=2)(x)
x=layers.Conv1D(filters=out_ch, kernel_size=3, padding="same", strides=1)(x)

x=layers.Dense(ecg_channels)(x)
x= layers.Dense(ecg_channels)(x)

autoencoder = keras.Model(input_thingy, x)
autoencoder.compile(optimizer=keras.optimizers.Adam(learning_rate=0.01), loss = 'mse')

autoencoder.summary()


Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_10 (InputLayer)        [(None, 1280, 23)]        0         
_________________________________________________________________
conv1d_64 (Conv1D)           (None, 1280, 46)          3220      
_________________________________________________________________
batch_normalization_42 (Batc (None, 1280, 46)          184       
_________________________________________________________________
max_pooling1d_21 (MaxPooling (None, 640, 46)           0         
_________________________________________________________________
conv1d_65 (Conv1D)           (None, 640, 92)           12788     
_________________________________________________________________
batch_normalization_43 (Batc (None, 640, 92)           368       
_________________________________________________________________
max_pooling1d_22 (MaxPooling (None, 320, 92)           0     

In [ ]:

#SEQUENTIAL

model = Sequential(name = 'Deep Convolutional Autoencoder')
time_length = 1280; ecg_channels = 23; in_ch = ecg_channels; out_ch = 46; network_depth = 3

model.add(Conv1D(filters=out_ch, kernel_size=3, padding="same", strides=1, input_shape=(time_length, in_ch)) )
for n in range(network_depth):
  model.add(keras.layers.ReLU())
  model.add(BatchNormalization(axis=-1))
  model.add(MaxPooling1D(pool_size = 2, padding = 'valid'))
  in_ch = out_ch;
  out_ch *= 2
  model.add(Conv1D(filters=out_ch, kernel_size=3, padding="same", strides=1))
  
  # Your encoded product is of size: Batch x Time_Length / (2**network_depth) x out_ch * 2**(network_depth-1)

out_ch = out_ch / 4

model.add(Conv1D(filters=out_ch, kernel_size=3, padding="same", strides=1))
for n in range(network_depth):
  model.add(keras.layers.ReLU())
  model.add(BatchNormalization(axis=-1))
  in_ch = out_ch;
  out_ch = out_ch // 2
  model.add(UpSampling1D(size=2))
  model.add(Conv1D(filters=out_ch, kernel_size=3, padding="same", strides=1))

model.add(keras.layers.Dense(ecg_channels))
model.add(keras.layers.Dense(ecg_channels))

model.compile(optimizer = keras.optimizers.Adam(learning_rate=0.01), loss = 'mse')
model.summary()

Model: "Deep Convolutional Autoencoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 1280, 46)          3220      
_________________________________________________________________
re_lu (ReLU)                 (None, 1280, 46)          0         
_________________________________________________________________
batch_normalization (BatchNo (None, 1280, 46)          184       
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 640, 46)           0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 640, 92)           12788     
_________________________________________________________________
re_lu_1 (ReLU)               (None, 640, 92)           0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 